In [1]:
import sys 
sys.path.append('..')

In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import dlib
import os, pickle
import re
import signal

## Apply FaceNet to an image

We'll use a pre-trained convolutional neural network on a subset of MS-Celebs-1M dataset (the dataset actually contains 100K celebs at this point). 

To generate the embedding we need to do the following steps:

1. Read an image
- Detect a face rectangle using Dlib face detector
- Crop the face, and possibly rotate and stretch it to fit the required dimensions of 160x160.
- Feed the cropped and aligned face image from the previous step into the network, and obtain the embedding.

## Detect face

In [4]:
from models.mtcnn.align_dlib import AlignDlib
align = AlignDlib('../models/dlib/shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()

def detect_face(path):
    image = mpimg.imread(path)
    bbs = detector(image, 1)
    tuples = []
    for r in bbs:
        tuples.append((r.left(), r.top(), r.right(), r.bottom()))
    return tuples, image

## Align face

In [5]:
EXPECT_SIZE = 160

def align_face(image, face_box, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE):
    assert isinstance(face_box, tuple)
    face_rect = dlib.rectangle(*face_box)
    landmarks = align.findLandmarks(image, face_rect)
    alignedFace = align.align(EXPECT_SIZE, image, face_rect, 
                              landmarks=landmarks,
                              landmarkIndices=landmarkIndices)
    return alignedFace

## Facenet Initialization Functions

https://github.com/davidsandberg/facenet

In [6]:
from os import path

def load_model(model_dir, model_meta, model_content):
    s = tf.InteractiveSession()
    model_dir_exp = os.path.expanduser(model_dir)
    saver = tf.train.import_meta_graph(os.path.join(model_dir_exp, meta_file))
    saver.restore(tf.get_default_session(), os.path.join(model_dir_exp, ckpt_file))
    tf.get_default_graph().as_graph_def()
    return s

In [7]:
def get_model_filenames(model_dir):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise ValueError('No meta file found in the model directory (%s)' % model_dir)
    elif len(meta_files)>1:
        raise ValueError('There should not be more than one meta file in the model directory (%s)' % model_dir)
    meta_file = meta_files[0]
    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return meta_file, ckpt_file

## Load Facenet

Pre trained model on MS-Celeb-1M

In [8]:
model_dir = '../models/facenet'
meta_file, ckpt_file = get_model_filenames(os.path.expanduser(model_dir))
session = load_model(model_dir, meta_file, ckpt_file)

In [9]:
graph = tf.get_default_graph()
image_batch = graph.get_tensor_by_name("input:0")
phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")
embeddings = graph.get_tensor_by_name("embeddings:0")

## Functions for Processing database

Load images in a database, calculate embeddings and save to file.

databse format: folders with name of the person, containing images of this person underneath

In [10]:
def rep_for_image_path(session, image_path):
    global image_batch, phase_train_placeholder, embeddings
    
    rects, face_orig = detect_face(image_path)
    if len(rects) > 0:
        face = align_face(face_orig, rects[0])
        feed_dict = { 
            image_batch: np.expand_dims(face, 0), 
            phase_train_placeholder: False }
    
        rep = session.run(embeddings, feed_dict=feed_dict)
    else:
        rep = None
    return rep

def reps_for_person(dir_name):
    all = []
    file_names = []

    for fname in os.listdir(dir_name):
        rep = rep_for_image_path(session, dir_name + '/' + fname)
        if rep is not None:
            all.append(rep[0])
            file_names.append(fname)
    return np.array(all), file_names

def load_people_faces(dir_path, person_paths, save_file):
    assert isinstance(person_paths, list)
    result = []
    all_file_names = []
    names = []
    i = 0
    rep_file = save_file + '_reps.npy'
    names_file = save_file + '_names.npy'
    print(rep_file, flush=True)
    if(os.path.exists(names_file) and os.path.exists(rep_file)):
        result = np.load(rep_file).tolist()
        names = np.load(names_file)
        i = len(result)
    for person_path in person_paths:
        if person_path in names:
            continue
        if(i%100 == 0):
            np.save(save_file + '_reps', result)
            np.save(save_file + '_names', person_paths[:i] )
        full_path = dir_path + '/' + person_path
        #print(full_path, flush=True)
        reps, file_names = reps_for_person(full_path)
        #print(len(reps), flush=True)
        print('Person ' + person_path + ' done.', flush=True)
        result.append(reps)
        all_file_names.append(file_names)
        i = i+1
    np.save(save_file + '_reps', result)
    np.save(save_file + '_names', person_paths[:i] )

def get_person_list(dir_name):
    return os.listdir(dir_name)

## Process Database 

Define paths

In [11]:
database_path = '../own_db'
save_path = '../models/facenet_own'

Get all folder names of persons contained in the dataset and then process all images. Save result to save_path.

In [12]:
names = get_person_list(database_path)
load_people_faces(database_path, names, save_path)

../models/facenet_own_reps.npy
Person Bill Gates done.
Person Britney Spears done.
Person Donald Trump done.
Person Fabian done.
Person Vladimir Putin done.


## Load calculated embeddings 

Loading pre-calculated embeddings on lfw:

In [14]:
names = np.load('../models/lfw_embeddings/facenet_names.npy')
people_reps = np.load('../models/lfw_embeddings/facenet_reps.npy')